In [5]:
# Parameters
n_cross_val = 6
fraction_training = 1.0
data_file = "../../data/experiments/pilot_random1_player_round_slim.csv"
data_dir = "../../data/artificial_humans/dev"
labels = {}
model_name = "graph"
job_id = 'dev'
model_args = {
    "y_levels": 31,
    "y_name": "punishments",
    "add_rnn": False,
    "add_edge_model": False,
    "add_global_model": False,
    "hidden_size": 10,
    "x_encoding": [
        {"name": "prev_contributions", "n_levels": 21, "encoding": "numeric"},
        {"name": "prev_punishments", "n_levels": 31, "encoding": "numeric"},
        {"name": "prev_valid", "etype": "bool"},
    ],
    "u_encoding": [
        {"name": "prev_common_good", "norm": 32, "etype": "float"},
        {"name": "round_number", "n_levels": 16, "encoding": "numeric"},
    ],
}
mask_name = "manager_valid"
experiment_names = ["trail_rounds_2"]
optimizer_args = {"lr": 0.001, "weight_decay": 1e-05}
train_args = {"epochs": 100, "batch_size": 20, "clamp_grad": 1, "eval_period": 10}
n_player = 4
shuffle_features = ["prev_punishments", "prev_contributions", "prev_common_good"]
device = "cpu"


In [6]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import torch as th
from aimanager.generic.data import create_syn_data, create_torch_data, get_cross_validations
from aimanager.artificial_humans import AH_MODELS
from aimanager.artificial_humans.evaluation import Evaluator
from aimanager.utils.array_to_df import using_multiindex
from aimanager.utils.utils import make_dir
from aimanager.generic.graph_encode import create_fully_connected
from torch_geometric.data import Batch
from torch_geometric.loader import DataLoader

model_dir = os.path.join(data_dir, 'model')
make_dir(model_dir)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
df = pd.read_csv(data_file)

df = df[df['experiment_name'].isin(experiment_names)]

data, default_values = create_torch_data(df)
# syn_data = create_syn_data(n_contribution=21, n_punishment=31, default_values=default_values)

In [8]:
th_device = th.device(device)

metrics = []
confusion_matrix = []
syn_pred = []
ev = Evaluator()

th_device = th.device(device)

syn_index = ['prev_punishments', 'prev_contributions']
edge_index = create_fully_connected(n_player)


def shuffle_feature(data, feature_name):
    data = {**data}
    data[feature_name] = data[feature_name][th.randperm(len(data[feature_name]))]
    return data

for i, (train_data, test_data) in enumerate(get_cross_validations(data, n_cross_val, fraction_training)):
    model = AH_MODELS[model_name](default_values=default_values, **model_args).to(th_device)

    train_data_ = model.encode(train_data, mask=mask_name, edge_index=edge_index)
    if test_data is not None:
        test_data_ = model.encode(test_data, mask=mask_name, edge_index=edge_index)
    # syn_data_ = model.encode(syn_data, mask=None, y_encode=False, info_columns=syn_index, edge_index=edge_index)

    # syn_df = using_multiindex(
    #     Batch.from_data_list(syn_data_)['info'].detach().cpu().numpy(), ['idx', 'round_number'], syn_index)

    optimizer = th.optim.Adam(model.parameters(), **optimizer_args)
    loss_fn = th.nn.CrossEntropyLoss(reduction='none')
    sum_loss = 0
    n_steps = 0

    for e in range(train_args['epochs']):
        ev.set_labels(cv_split=i, epoch=e)
        model.train()
        for j, batch_data in enumerate(iter(DataLoader(train_data_, shuffle=True, batch_size=train_args['batch_size']))):
            optimizer.zero_grad()
            py = model(batch_data).flatten(end_dim=-2)
            y_true = batch_data['y_enc'].flatten(end_dim=-2)
            loss = loss_fn(py, y_true)
            mask = batch_data['mask'].flatten()
            loss = (loss * mask).sum() / mask.sum()

            loss.backward()

            if train_args['clamp_grad']:
                for param in model.parameters():
                    param.grad.data.clamp_(-train_args['clamp_grad'], train_args['clamp_grad'])
            optimizer.step()
            sum_loss += loss.item()
            n_steps +=1
        
        last_epoch = e == (train_args['epochs'] - 1)

        if (e % train_args['eval_period'] == 0) or last_epoch:
            avg_loss = sum_loss/n_steps
            print(f'CV {i} | Epoch {e} | Loss {avg_loss}')
            ev.add_loss(avg_loss)

            ev.eval_set(model, train_data_, calc_confusion=last_epoch, set='train')
            if test_data is not None:
                ev.eval_set(model, test_data_, calc_confusion=last_epoch, set='test')
                for sf in shuffle_features:
                    shuffled_data = shuffle_feature(test_data, sf)
                    shuffled_data = model.encode(shuffled_data, mask=mask_name, edge_index=edge_index)
                    ev.eval_set(model, shuffled_data, calc_confusion=False, set='test', shuffle_feature=sf)
            sum_loss = 0
            n_steps = 0
    # ev.eval_syn(model, syn_data_, syn_df)


ev.save(data_dir, labels, job_id=job_id)
model_path = os.path.join(model_dir, f'{job_id}.pt')
model.save(model_path)

CV 0 | Epoch 0 | Loss 3.223150849342346
CV 0 | Epoch 10 | Loss 3.122298848628998
CV 0 | Epoch 20 | Loss 2.8920508325099945
CV 0 | Epoch 30 | Loss 2.537504267692566
CV 0 | Epoch 40 | Loss 2.1290676176548002
CV 0 | Epoch 50 | Loss 1.8018991976976395
CV 0 | Epoch 60 | Loss 1.6644995599985122
CV 0 | Epoch 70 | Loss 1.6126536816358565
CV 0 | Epoch 80 | Loss 1.5936220288276672
CV 0 | Epoch 90 | Loss 1.590122902393341
CV 0 | Epoch 99 | Loss 1.5727310809824202
CV 1 | Epoch 0 | Loss 3.4140405654907227
CV 1 | Epoch 10 | Loss 3.291119408607483
CV 1 | Epoch 20 | Loss 3.0707785964012144
CV 1 | Epoch 30 | Loss 2.8067487835884095
CV 1 | Epoch 40 | Loss 2.4663821697235107
CV 1 | Epoch 50 | Loss 2.059183123707771
CV 1 | Epoch 60 | Loss 1.8015556573867797
CV 1 | Epoch 70 | Loss 1.7212061405181884
